In [1]:
import datetime
import h5py
import numpy as np
import os
import pandas as pd
import sys
import time

sys.path.append('../src')
import localmodule


# Define constants.
args = ["unit01", "thrush"] #                            DISABLE
#args = sys.argv[1:]                                     ENABLE
unit_str = args[0]
odf_str = args[1]
data_dir = localmodule.get_data_dir()
dataset_name = localmodule.get_dataset_name()
models_dir = localmodule.get_models_dir()
oldbird_models_dir = os.path.join(models_dir, "oldbird")
unit_dir = os.path.join(oldbird_models_dir, unit_str)


# Define input and output folder.
in_predictions_name = "_".join(["predictions", "no-clip-suppressor"])
in_predictions_dir = os.path.join(unit_dir, in_predictions_name)
out_predictions_name = "_".join(["predictions", "clip-suppressor"])
out_predictions_dir = os.path.join(unit_dir, out_predictions_name)
os.makedirs(out_predictions_dir, exist_ok=True)


# Define suppressor count threshold and period.
oldbird_data_name = "_".join([dataset_name, "oldbird"])
oldbird_data_dir = os.path.join(data_dir, oldbird_data_name)
oldbird_data_path = os.path.join(oldbird_data_dir, unit_str + ".hdf5")
oldbird_hdf5 = h5py.File(oldbird_data_path, "r")
settings_key = "_".join([odf_str, "settings"])
settings = oldbird_hdf5[settings_key]
suppressor_count_threshold = settings["suppressor_count_threshold"].value
suppressor_period = settings["suppressor_period"].value

In [23]:
n_thresholds = 100
threshold_id = 0

# for threshold_id in range(n_thresholds):
threshold_str = "th-" + str(threshold_id).zfill(2)
in_prediction_name = "_".join(
    [dataset_name, "oldbird", odf_str,
     unit_str, threshold_str, "predictions.csv"])
in_prediction_path = os.path.join(in_predictions_dir, in_prediction_name)
out_prediction_name = "_".join(
    [dataset_name, "oldbird", odf_str,
     unit_str, threshold_str, "predictions_clip-suppressor.csv"])
out_prediction_path = os.path.join(out_predictions_dir, out_prediction_name)

in_df = pd.read_csv(in_prediction_path)
in_times = np.array(in_df["Time (s)"])

n_rows = len(in_times)
n = 0
out_times = []

while n < (n_rows-suppressor_count_threshold):
    current_time = in_times[n]
    next_n = n + suppressor_count_threshold
    next_time = in_times[next_n]
    time_difference = next_time - current_time
    if time_difference < suppressor_period:
        while time_difference < suppressor_period:
            next_n = next_n + 1
            next_time = in_times[next_n]
            time_difference = next_time - current_time
        n = next_n
    else:
        times.append(current_time)
        n = n + 1

out_df = in_df[in_df["Time (s)"].isin(times)]

out_df.to_csv(out_prediction_path)